<a href="https://colab.research.google.com/github/meyresearch/ANI-Peptides/blob/main/demos/AMBER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime Environment Setup

Before you start, ensure you have been allocated a GPU runtime. Go to `Runtime -> Change runtime type -> GPU`

Allow kernel to restart after running this cell.

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:40
🔁 Restarting kernel...


Ensure good conda/mamba installation

In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


Install Dependencies



*   *OpenMM* - Molecular dynamics simulation
*   *NGLView, PyTraj* - Viewing libraries



In [2]:
!mamba install -q openmm nglview
!pip install -q pytraj

  Package                          Version  Build                Channel                    Size
──────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────

  argcomplete                       1.12.3  pyhd8ed1ab_2         conda-forge/noarch        34 KB
  argon2-cffi                       21.1.0  py37h5e8e339_2       conda-forge/linux-64      46 KB
  async_generator                     1.10  py_0                 conda-forge/noarch        18 KB
  attrs                             21.2.0  pyhd8ed1ab_0         conda-forge/noarch        44 KB
  backcall                           0.2.0  pyh9f0ad1d_0         conda-forge/noarch        13 KB
  backports                            1.0  py_2                 conda-forge/noarch         4 KB
  backports.functools_lru_cache      1.6.4  pyhd8ed1ab_0         conda-forge/noarch         9 KB
  bleach      

Test OpenMM

In [3]:
!python -m openmm.testInstallation


OpenMM Version: 7.6
Git Revision: ad113a0cb37991a2de67a08026cf3b91616bafbe

There are 4 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 CUDA - Successfully computed forces
4 OpenCL - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.29783e-06
Reference vs. CUDA: 6.72816e-06
CPU vs. CUDA: 7.53579e-07
Reference vs. OpenCL: 6.76359e-06
CPU vs. OpenCL: 8.13039e-07
CUDA vs. OpenCL: 2.62645e-07

All differences are within tolerance.


Fetch sample peptide

In [4]:
!wget -q https://github.com/meyresearch/ANI-Peptides/raw/main/demos/aaa.pdb

# AMBER Simulation

In [ ]:
# Import libraries

from openmm.app import *
from openmm import *
from openmm.unit import *

In [6]:
# Progress bar for simulations

from IPython.display import ProgressBar

class ProgressBarWriter(ProgressBar):
    def write(self, string):
        if string.startswith("#"):
            self.progress = 0
        else:
            string = string.strip(',\n')
            try:
                self.progress = int(string)
            except ValueError:
                pass

def ProgressBarReporter(steps, interval):
    return StateDataReporter(
        ProgressBarWriter(total = steps), 
        interval, 
        step = True,
        totalSteps = steps,
    )

In [16]:
# Simulation function (to run simulations consistently)

def simulate(
    simulation,
    output_pdb_filename: str = "output.pdb",
    output_state_data_filename: str = "state_data.csv",
    step_size: Quantity = 0.004*femtoseconds,
    steps: int or None = 100000,
    duration: Quantity or None = None,
    pdb_report_interval: int or None = None,
    state_report_interval: int or None = None,
    progressbar_interval: int or None = None,
):
    assert duration or steps

    if duration:
        steps = int(duration / step_size)

    if not pdb_report_interval:
        pdb_report_interval = steps // 10

    if not state_report_interval:
        state_report_interval = steps // 10

    if not progressbar_interval:
        progressbar_interval = steps // 50

    # Create pdb file reporter
    simulation.reporters.append(
        PDBReporter(
            output_pdb_filename,
            pdb_report_interval
        )
    )

    # Create state data file reporter
    simulation.reporters.append(
        StateDataReporter(
            output_state_data_filename, 
            state_report_interval, 
            step=True,
            potentialEnergy=True, 
            temperature=True
        )
    )

    # Create progress bar reporter
    simulation.reporters.append(
        ProgressBarReporter(steps, progressbar_interval)
    )

    # Run simulation through specified time steps
    print("Running simulation...")
    simulation.step(steps)

In [17]:
# Equilibration function - Constant pressure & temp

def equilibrate(
    coords: Topology,
    forcefield: ForceField,
    pressure: Quantity = 1*atmosphere,
    temperature: Quantity = 300*kelvin,
    friction_coeff: Quantity = 1/femtosecond,
    step_size: Quantity = 0.004*femtoseconds,
    **kwargs
):      
    print("Initialising equilibration run...")

    # Create system
    system = forcefield.createSystem(
        coords.topology, 
        nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer,
        constraints=AllBonds,
        hydrogenMass=4*amu,
    )

    # Add a barostat for constant pressure
    system.addForce(
        MonteCarloBarostat(
            pressure,
            temperature
        )
    )

    # Create constant temp integrator
    integrator = LangevinMiddleIntegrator(
        temperature,
        friction_coeff,
        step_size
    )

    # Create simulation and set initial positions
    simulation = Simulation(
        coords.topology,
        system,
        integrator
    )
    simulation.context.setPositions(coords.positions)

    # Local energy minimisation
    print("Local energy minimisation...")
    simulation.minimizeEnergy()

    simulate(
        simulation,
        **kwargs
    )

    return simulation

In [18]:
# Production function - Constant volume & temp

def production(
    coords: Topology,
    forcefield: ForceField,
    temperature: Quantity = 300*kelvin,
    friction_coeff: Quantity = 1/femtosecond,
    step_size: Quantity = 0.004*femtoseconds,
    **kwargs
):      
    print("Initialising production run...")

    # Create system
    system = forcefield.createSystem(
        coords.topology, 
        nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer,
        constraints=AllBonds,
        hydrogenMass=4*amu,
    )

    # Create constant temp integrator
    integrator = LangevinMiddleIntegrator(
        temperature,
        friction_coeff,
        step_size
    )

    # Create simulation and set initial positions
    simulation = Simulation(
        coords.topology,
        system,
        integrator
    )
    simulation.context.setPositions(coords.positions)

    simulate(
        simulation,
        **kwargs
    )

    return simulation

In [19]:
# Load sample peptide
pdb = PDBFile('aaa.pdb')

# Create AMBER forcefield
forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3pfb.xml'
)

# Load pdb into modeller and add solvent
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)

simulation = equilibrate(
    modeller,
    forcefield,
    pressure = 1*atmosphere,
    temperature = 300*kelvin,
    friction_coeff = 1/femtosecond,
    step_size = 0.004*femtoseconds,
    steps = 10000,
    output_pdb_filename = "equilibration_output.pdb",
    output_state_data_filename = "equilibration_state_data.csv"
)

simulation.positions = simulation.context.getState(getPositions=True).getPositions()

production(
    simulation,
    forcefield,
    temperature = 300*kelvin,
    friction_coeff = 1/femtosecond,
    step_size = 0.004*femtoseconds,
    steps = 10000,
    output_pdb_filename = "equilibration_output.pdb",
    output_state_data_filename = "equilibration_state_data.csv"
)

Initialising equilibration run...
Local energy minimisation...
Running simulation...


[============================================================] 10000/10000

Initialising production run...
Running simulation...


[============================================================] 10000/10000

In [ ]:
# Load sample peptide
coords = PDBFile('aaa.pdb')






# Create AMBER forcefield
forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3pfb.xml'
)

# Create system
system = forcefield.createSystem(
    pdb.topology, 
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=HBonds
)

# Create constant temp integrator
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.004*picoseconds
)

# Create simulation and set initial positions
simulation = Simulation(
    pdb.topology,
    system,
    integrator
)
simulation.context.setPositions(pdb.positions)

# Local energy minimisation
simulation.minimizeEnergy()

# Create reporters for output
simulation.reporters.append(
    PDBReporter('output.pdb', 1000)
)
simulation.reporters.append(
    StateDataReporter(
        stdout, 
        1000, 
        step=True,
        potentialEnergy=True, 
        temperature=True
    )
)

In [ ]:
# Run the simulation through 10000 time steps
simulation.step(10000)

# Viewer

In [ ]:
import pytraj as pt
import nglview as nv
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
traj = pt.load("output.pdb")
view = nv.show_pytraj(traj)
view.add_representation("licorice")
view.add_unitcell()
view